<a href="https://colab.research.google.com/github/SOUMEE2000/Machine-Learning-Stash/blob/main/7.%20Digit%20Classifiers/Using_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [3]:
def build_generator():
  noise_shape = (100,)
  model = Sequential()

  model.add(Dense(256, input_shape=noise_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
    
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()

  noise = Input(shape=noise_shape)
  img = model(noise)    #Generated image

  return Model(noise, img)

In [4]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [5]:
(X_train, _), (_, _) = mnist.load_data()

# Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
X_train = np.expand_dims(X_train, axis=3) 
batch_size = 32
half_batch = int(batch_size / 2)


11501568/11490434 [==============================] - 0s 0us/step


In [6]:
half_batch

16

In [7]:
X_train.shape

(60000, 28, 28, 1)

In [8]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("/content/images/mnist_%d.png" % epoch)
    plt.close()

In [9]:
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3) 
    half_batch = int(batch_size / 2) 
    
    for epoch in range(epochs):

        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

 
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
  
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size
        g_loss = combined.train_on_batch(noise, valid_y)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [11]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)
discriminator.trainable = False  
valid = discriminator(img) 
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)
train(epochs=100000, batch_size=32, save_interval=1000)


Streaming output truncated to the last 5000 lines.
17944 [D loss: 0.713249, acc.: 56.25%] [G loss: 0.755505]
17945 [D loss: 0.698360, acc.: 53.12%] [G loss: 0.841687]
17946 [D loss: 0.698695, acc.: 62.50%] [G loss: 0.725540]
17947 [D loss: 0.720914, acc.: 53.12%] [G loss: 0.727428]
17948 [D loss: 0.605904, acc.: 75.00%] [G loss: 0.771625]
17949 [D loss: 0.730729, acc.: 50.00%] [G loss: 0.845186]
17950 [D loss: 0.692223, acc.: 62.50%] [G loss: 0.855098]
17951 [D loss: 0.687331, acc.: 53.12%] [G loss: 0.876574]
17952 [D loss: 0.742346, acc.: 40.62%] [G loss: 0.894747]
17953 [D loss: 0.736337, acc.: 50.00%] [G loss: 0.880839]
17954 [D loss: 0.683015, acc.: 53.12%] [G loss: 0.880118]
17955 [D loss: 0.690724, acc.: 59.38%] [G loss: 0.909646]
17956 [D loss: 0.648360, acc.: 75.00%] [G loss: 0.885699]
17957 [D loss: 0.631440, acc.: 62.50%] [G loss: 0.871376]
17958 [D loss: 0.641203, acc.: 56.25%] [G loss: 0.893003]
17959 [D loss: 0.647713, acc.: 56.25%] [G loss: 0.809527]
17960 [D loss: 0.6774

KeyboardInterrupt: ignored

In [12]:
generator.save('/content/generator_model.h5')
